In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [ ]:


# load dataset and split
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# model hyperparameters
params = {
    "penalty": "l2",
    "solver": "lbfgs",
    "max_iter": 100,
    "multi_class": "auto",
    "random_state": 8888,
}

In [ ]:
# configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("first ML experiment")

# train
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# predict & evaluate
y_pred = lr.predict(X_test)
accuracy = float(accuracy_score(y_test, y_pred))

In [ ]:
# log run to MLflow
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.set_tag("training info", "basic LR model for iris data")

    # infer signature and log model (register if registry available)
    signature = infer_signature(X_train, lr.predict(X_train))
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

print(f"Accuracy: {accuracy:.4f}")
print(f"Model URI: {model_info.model_uri}")